🔗 [Back to Table of Contents](https://github.com/najaeda/najaeda-tutorials#-table-of-contents)
# Chapter 1: Getting Started with najaeda

Welcome to **najaeda**! This guide will help you quickly install and start using the `najaeda` Python package.

## Installation



First, let's clone najaeda-tutorials repository to access the tutorials files.

In [1]:
!git clone https://github.com/najaeda/najaeda-tutorials.git

Clonage dans 'najaeda-tutorials'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 1), reused 13 (delta 1), pack-reused 0 (from 0)
Réception d'objets: 100% (17/17), 1.86 Mio | 1.10 Mio/s, fait.
Résolution des deltas: 100% (1/1), fait.


To install `najaeda`, simply use pip:

In [2]:
pip install najaeda

Note: you may need to restart the kernel to use updated packages.


## Import najaeda
Now we will import najaeda needed package:

In [3]:
from najaeda import netlist

---
✅ [Next Chapter → Basic Usage](https://colab.research.google.com/github/najaeda/najaeda-tutorials/blob/main/notebooks/02_basic_usage.ipynb)